<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/SQL_Options_in_Spark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

     |████████████████████████████████| 212.4MB 71kB/s 
     |████████████████████████████████| 204kB 18.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=8dbd9ebe37000ab159f50ecb65703a6826f25303ea256ac80203b2f0400e0dba
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
You are working with 1 core(s)


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
googlep = spark.read.csv(path + 'googleplaystore.csv', inferSchema=True, header=True)

Mounted at /content/drive


## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [3]:
googlep.show(5, False)

+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-------------------------+----------------+------------------+------------+
|App                                               |Category      |Rating|Reviews|Size|Installs   |Type|Price|Content Rating|Genres                   |Last Updated    |Current Ver       |Android Ver |
+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-------------------------+----------------+------------------+------------+
|Photo Editor & Candy Camera & Grid & ScrapBook    |ART_AND_DESIGN|4.1   |159    |19M |10,000+    |Free|0    |Everyone      |Art & Design             |January 7, 2018 |1.0.0             |4.0.3 and up|
|Coloring book moana                               |ART_AND_DESIGN|3.9   |967    |14M |500,000+   |Free|0    |Everyone      |Art & Design;Pretend Play|January 15, 2018|2.0.0             |4.0.3 and

In [4]:
# This is way better
googlep.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

In [5]:
print(googlep.printSchema())

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [8]:
from pyspark.sql.types import IntegerType, FloatType
df = googlep.withColumn("Rating", googlep["Rating"].cast(FloatType())) \
            .withColumn("Reviews", googlep["Reviews"].cast(IntegerType())) \
            .withColumn("Price", googlep["Price"].cast(IntegerType()))
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [9]:
df.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [13]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").show(5,False)

+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-----------------------+------------------+------------------+------------+
|App                                               |Category      |Rating|Reviews|Size|Installs   |Type|Price|Content Rating|Genres                 |Last Updated      |Current Ver       |Android Ver |
+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-----------------------+------------------+------------------+------------+
|U Launcher Lite – FREE Live Cool Themes, Hide Apps|ART_AND_DESIGN|4.7   |87510  |8.7M|5,000,000+ |Free|0    |Everyone      |Art & Design           |August 1, 2018    |1.2.4             |4.0.3 and up|
|Sketch - Draw & Paint                             |ART_AND_DESIGN|4.5   |215644 |25M |50,000,000+|Free|0    |Teen          |Art & Design           |June 8, 2018      |Varies with device|4.2 and u

In [12]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").select(['Rating', 'App']).show(5,False)

+------+--------------------------------------------------+
|Rating|App                                               |
+------+--------------------------------------------------+
|4.7   |U Launcher Lite – FREE Live Cool Themes, Hide Apps|
|4.5   |Sketch - Draw & Paint                             |
|4.3   |Pixel Draw - Number Art Coloring Book             |
|4.4   |Paper flowers instructions                        |
|4.4   |Garden Coloring Book                              |
+------+--------------------------------------------------+
only showing top 5 rows



## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [15]:
sql_results = spark.sql("SELECT App, Rating FROM tempview WHERE Category = 'COMICS' AND Rating > 4.5")
sql_results.show(5,False)

+--------------------------------------------------------------------+------+
|App                                                                 |Rating|
+--------------------------------------------------------------------+------+
|Manga Master - Best manga & comic reader                            |4.6   |
|GANMA! - All original stories free of charge for all original comics|4.7   |
|Röhrich Werner Soundboard                                           |4.7   |
|Unicorn Pokez - Color By Number                                     |4.8   |
|Manga - read Thai translation                                       |4.6   |
+--------------------------------------------------------------------+------+
only showing top 5 rows



## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [17]:
spark.sql("SELECT Category, sum(Reviews) AS Total_Reviews FROM tempview GROUP BY Category ORDER BY Total_Reviews DESC").show(1)

+--------+-------------+
|Category|Total_Reviews|
+--------+-------------+
|    GAME|   1585422349|
+--------+-------------+
only showing top 1 row



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [19]:
spark.sql("SELECT App, Reviews FROM tempview ORDER BY Reviews DESC").show(1)

+--------+--------+
|     App| Reviews|
+--------+--------+
|Facebook|78158306|
+--------+--------+
only showing top 1 row



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [22]:
spark.sql("SELECT App, Reviews \
               FROM tempview \
               WHERE App Like '%dating%' \
                                           ").show(5,False)

+--------------------------------------------------+-------+
|App                                               |Reviews|
+--------------------------------------------------+-------+
|Meet, chat & date. Free dating app - Chocolate app|8661   |
|Friend Find: free chat + flirt dating app         |23     |
|Spine- The dating app                             |5      |
|Princess Closet : Otome games free dating sim     |29495  |
|happn – Local dating app                          |1118201|
+--------------------------------------------------+-------+



## 6. Use SQL Transformer to display how many free apps there are in this list

In [24]:
from pyspark.ml.feature import SQLTransformer

sqlTrans = SQLTransformer(statement = "SELECT count(*) FROM __THIS__ WHERE Type = 'Free'")
sqlTrans.transform(df).show()

+--------+
|count(1)|
+--------+
|   10037|
+--------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [25]:
sqlTrans = SQLTransformer(statement = "SELECT Genres, count(*) as Total FROM __THIS__ GROUP BY Genres ORDER BY Total DESC")
sqlTrans.transform(df).show(5)

+-------------+-----+
|       Genres|Total|
+-------------+-----+
|        Tools|  842|
|Entertainment|  623|
|    Education|  549|
|      Medical|  463|
|     Business|  460|
+-------------+-----+
only showing top 5 rows



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [33]:
sqlTrans = SQLTransformer(statement = "SELECT App, Genres, Reviews FROM __THIS__ WHERE Genres = 'Tools' AND Reviews > 100 ORDER BY Reviews DESC")
sqlTrans.transform(df).show(5,False)

+--------------------------------------------------+------+--------+
|App                                               |Genres|Reviews |
+--------------------------------------------------+------+--------+
|Clean Master- Space Cleaner & Antivirus           |Tools |42916526|
|Security Master - Antivirus, VPN, AppLock, Booster|Tools |24900999|
|360 Security - Free Antivirus, Booster, Cleaner   |Tools |16771865|
|DU Battery Saver - Battery Charger & Battery Life |Tools |13479633|
|Cache Cleaner-DU Speed Booster (booster & cleaner)|Tools |12759815|
+--------------------------------------------------+------+--------+
only showing top 5 rows



## That's all folks! Great job!